<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-31 13:22:26
-------------------
qualified stocks: 95
with latest results: 30
still star stocks: 20
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  4.51 L
Current:  1.37 C
-------------------
Today PnL: -23.57 K (-0.17%)
Current PnL: -17.15 L (-11.66%)
CY Booked + Current PnL: -9.12 L (-6.2%)
-------------------
Total profit:  3.36 L
Total loss:  -20.50 L
-------------------
Total Booked + Current PnL: 17.86 L (14.88%)
Total Booked PnL: 35.01 L (29.16%)
Curr Year Booked PnL: 8.03 L (5.84%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 75.38 L (54.83%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.98%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,HINDALCO,651.95,761.55,-19.90,H-LC,5.32,110456.0,6144.0,11388.0,0.23,5.89,10.31,16.81,10.0,0.54,0.83,22.88,X5K,ATH,METALS
17,BRITANNIA,5190.70,6446.05,16.31,H-LC,9.70,104400.0,10967.0,11630.0,0.92,11.74,11.14,24.18,31.0,0.94,0.79,26.77,XY25,ATH,FMCG
4,APOLLOHOSP,6904.43,8285.00,-18.07,H-LC,6.56,111675.0,8109.0,12597.0,-0.07,7.83,11.28,20.00,15.0,0.64,0.84,23.00,X40N,BTT,HEALTHCARE
74,SIS,477.00,477.00,2341.45,H-SC,6.61,65688.0,-15402.0,15404.0,0.57,-18.99,23.45,0.00,157.0,-1.00,0.49,33.13,OX40N,NTT,MISC
70,SBIN,760.30,863.00,-12.46,M-LC,4.30,212570.0,11090.0,16134.0,0.06,5.50,7.59,13.51,62.0,0.69,1.60,18.79,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,QUESS,296.83,986.00,-34.44,M-SC,41.81,65207.0,201.0,150726.0,-0.02,0.31,231.15,232.18,199.0,0.00,0.49,1.31,XY24,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,9.11,H-SC,19.82,101523.0,560.0,93970.0,4.43,0.55,92.56,93.63,149.0,0.01,0.76,51.22,SR,ATH,CHEMICALS
38,HEROMOTOCO,4311.81,5949.07,-6.67,H-MC,3.54,150048.0,-865.0,58174.0,0.84,-0.57,38.77,37.97,90.0,-0.01,1.13,24.59,AR,ATH,AUTO


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.0,-19.44,H-LC,1.72,212254.0,7754.0,33663.0,-0.65,3.79,15.86,20.25,9.0,0.23,1.60,12.86,X40N,BTT,PHARMA
10,BAJAJHFL,122.26,152.0,-9.80,H-LC,2.48,187157.0,-13349.0,62117.0,-0.77,-6.66,33.19,24.33,30.0,-0.21,1.41,7.93,X40N,BTT,FINANCE
56,LTIM,5564.16,7230.2,-4.21,H-LC,2.67,184302.0,-16008.0,75988.0,-0.39,-7.99,41.23,29.94,24.0,-0.21,1.39,27.33,X200,ATH,IT
5,ASIANPAINT,2961.56,4250.0,-17.68,H-LC,3.24,163526.0,-37860.0,125473.0,-0.46,-18.80,76.73,43.51,19.0,-0.30,1.23,13.83,X40,BTT,PAINTS
40,HINDUNILVR,2413.81,2723.0,-9.25,H-LC,4.26,284010.0,13663.0,20960.0,4.04,5.05,7.38,12.81,28.0,0.65,2.14,18.78,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,2347.98,4546.37,9.11,H-SC,19.82,101523.0,560.0,93970.0,4.43,0.55,92.56,93.63,149.0,0.01,0.76,51.22,SR,ATH,CHEMICALS
40,HINDUNILVR,2413.81,2723.00,-9.25,H-LC,4.26,284010.0,13663.0,20960.0,4.04,5.05,7.38,12.81,28.0,0.65,2.14,18.78,XY25,NTT,FMCG
68,SAMMAANCAP,170.35,326.00,-167.05,M-SC,4.22,77310.0,-24900.0,118292.0,4.01,-24.36,153.01,91.37,210.0,-0.21,0.58,25.50,XY25,NTT,FINANCE
51,JIOFIN,310.58,387.00,8.02,H-LC,13.32,231560.0,14154.0,39342.0,3.28,6.51,16.99,24.61,50.0,0.36,1.74,64.64,XY24,BTT,FINANCE
88,UNITDSPR,1376.96,1644.00,-11.80,H-LC,7.42,177448.0,-4311.0,39553.0,2.17,-2.37,22.29,19.39,11.0,-0.11,1.33,4.95,X40N,NTT,BREWERIES


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,GREENPANEL,375.16,537.00,219.77,M-SC,8.41,142838.0,-40240.0,119213.0,-8.52,-21.98,83.46,43.14,232.0,-0.34,1.07,34.36,XY24,NTT,MISC
64,RELAXO,902.64,1176.00,-34.93,H-SC,13.73,58803.0,-56735.0,91727.0,-4.88,-49.11,155.99,30.28,134.0,-0.62,0.44,14.88,X40N,NTT,FOOTWEAR
18,BSOFT,628.85,836.99,-16.70,H-SC,12.44,70096.0,-39953.0,76377.0,-3.42,-36.30,108.96,33.10,129.0,-0.52,0.53,16.48,XR,ATH,IT
20,CAMS,3643.00,5226.82,-6.14,H-SC,0.25,105330.0,3326.0,41016.0,-3.30,3.26,38.94,43.48,120.0,0.08,0.79,20.59,X40N,ATH,MISC
93,WHIRLPOOL,1219.98,2270.00,-40.07,M-SC,7.50,100868.0,9370.0,69387.0,-2.82,10.24,68.79,86.07,211.0,0.14,0.76,45.92,XR,NTT,DURABLES


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,SYMPHONY,1306.42,1306.0,-30.51,M-SC,8.86,148030.0,-23111.0,23063.0,-0.88,-13.50,15.58,-0.03,197.0,-1.00,1.11,7.12,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,6.15,149011.0,-25548.0,25585.0,-0.09,-14.64,17.17,0.02,135.0,-1.00,1.12,29.22,OX40N,NTT,PAINTS
86,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.86,118989.0,-23490.0,24107.0,0.99,-16.49,20.26,0.44,163.0,-0.97,0.89,40.44,OX40N,NTT,BANKS
80,TATAELXSI,7332.28,7332.0,-18.82,H-MC,1.85,85260.0,-17392.0,17385.0,-0.18,-16.94,20.39,-0.00,101.0,-1.00,0.64,29.61,OX40N,NTT,IT
21,CERA,7989.07,8422.0,-17.42,H-SC,2.90,117918.0,-25885.0,33677.0,0.58,-18.00,28.56,5.42,155.0,-0.77,0.89,30.13,OX40N,NTT,CERAMICS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TCS,3794.03,4998.00,-27.06,H-LC,13.44,237284.0,-58650.0,152550.0,-0.38,-19.82,64.29,31.73,1.0,-0.38,1.78,0.00,X40,BTT,IT
32,ENRIN,1377.95,3258.00,NaN,NaN,2.89,81450.0,47001.0,0.0,0.96,136.44,0.00,136.44,2.0,inf,0.61,24.55,AR,ATH,ELECTRICAL
91,VBL,492.64,672.28,-7.47,H-LC,11.92,280944.0,15904.0,80743.0,0.02,6.00,28.74,36.46,3.0,0.20,2.11,19.98,X40N,ATH,FMCG
48,INFY,1461.46,2275.00,-15.29,H-LC,7.11,271310.0,9709.0,135926.0,-0.22,3.71,50.10,55.67,4.0,0.07,2.04,10.04,X40,BTT,IT
1,ABB,5551.76,7934.00,-39.28,H-LC,7.48,216704.0,185.0,92728.0,0.05,0.09,42.79,42.91,5.0,0.00,1.63,13.27,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,4.23,52100.0,10845.0,6591.0,0.27,26.29,12.65,42.26,198.0,1.65,0.39,22.66,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,19.04,H-SC,12.10,152586.0,26386.0,19790.0,0.70,20.91,12.97,36.59,156.0,1.33,1.15,36.04,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3258.00,NaN,NaN,2.89,81450.0,47001.0,0.0,0.96,136.44,0.00,136.44,2.0,inf,0.61,24.55,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,KPIGREEN,497.21,731.64,27.90,H-SC,12.86,130750.0,5453.0,53621.0,0.80,4.35,41.01,47.15,144.0,0.10,0.98,63.58,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,19.04,H-SC,12.10,152586.0,26386.0,19790.0,0.70,20.91,12.97,36.59,156.0,1.33,1.15,36.04,XY24,NTT,HEALTHCARE
3,ALKYLAMINE,2347.98,4546.37,9.11,H-SC,19.82,101523.0,560.0,93970.0,4.43,0.55,92.56,93.63,149.0,0.01,0.76,51.22,SR,ATH,CHEMICALS
26,DABUR,505.20,735.00,-4.85,H-MC,4.92,205939.0,8911.0,80707.0,1.16,4.52,39.19,45.49,100.0,0.11,1.55,18.15,XY24,BTT,FMCG
20,CAMS,3643.00,5226.82,-6.14,H-SC,0.25,105330.0,3326.0,41016.0,-3.30,3.26,38.94,43.48,120.0,0.08,0.79,20.59,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3258.00,NaN,NaN,2.89,81450.0,47001.0,0.0,0.96,136.44,0.00,136.44,2.0,inf,0.61,24.55,AR,ATH,ELECTRICAL
93,WHIRLPOOL,1219.98,2270.00,-40.07,M-SC,7.50,100868.0,9370.0,69387.0,-2.82,10.24,68.79,86.07,211.0,0.14,0.76,45.92,XR,NTT,DURABLES
45,INDIAMART,2327.09,4850.92,-47.14,H-SC,13.18,137662.0,14326.0,119436.0,0.47,11.62,86.76,108.45,118.0,0.12,1.04,36.57,AR,ATH,MISC
53,KPIGREEN,497.21,731.64,27.90,H-SC,12.86,130750.0,5453.0,53621.0,0.80,4.35,41.01,47.15,144.0,0.10,0.98,63.58,MH,ATH,POWER
94,WIPRO,243.46,420.00,-9.67,M-LC,5.54,154504.0,3559.0,105897.0,-0.40,2.36,68.54,72.51,55.0,0.03,1.16,8.52,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,7.00,19852.0,-29741.0,941.0,-1.09,-59.97,4.74,-58.07,261.0,-31.61,0.15,33.13,XY24,ATH,MISC
74,SIS,477.00,477.00,2341.45,H-SC,6.61,65688.0,-15402.0,15404.0,0.57,-18.99,23.45,0.00,157.0,-1.00,0.49,33.13,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.20,70099.0,-43450.0,83502.0,0.86,-38.27,119.12,35.27,267.0,-0.52,0.53,71.57,XR,NTT,HOTELS
32,ENRIN,1377.95,3258.00,NaN,NaN,2.89,81450.0,47001.0,0.0,0.96,136.44,0.00,136.44,2.0,inf,0.61,24.55,AR,ATH,ELECTRICAL
57,MASFIN,326.60,399.50,-17.80,H-SC,16.87,93240.0,-4740.0,26611.0,0.91,-4.84,28.54,22.32,151.0,-0.18,0.70,35.23,XR,ATH,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,4.23,52100.0,10845.0,6591.0,0.27,26.29,12.65,42.26,198.0,1.65,0.39,22.66,XY24,ATH,IT
16,BLUSPRING,226.45,94.95,NaN,NaN,7.00,19852.0,-29741.0,941.0,-1.09,-59.97,4.74,-58.07,261.0,-31.61,0.15,33.13,XY24,ATH,MISC
58,MEDANTA,1087.93,1486.00,19.04,H-SC,12.10,152586.0,26386.0,19790.0,0.70,20.91,12.97,36.59,156.0,1.33,1.15,36.04,XY24,NTT,HEALTHCARE
91,VBL,492.64,672.28,-7.47,H-LC,11.92,280944.0,15904.0,80743.0,0.02,6.00,28.74,36.46,3.0,0.20,2.11,19.98,X40N,ATH,FMCG
29,DIXON,14367.00,18940.15,-26.50,H-MC,5.82,135320.0,20384.0,16198.0,0.89,17.74,11.97,31.83,85.0,1.26,1.02,52.29,X40N,ATH,IT


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.78
1,25,41.69
2,50,70.73


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.98
LC    35.64
MC    22.19
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.73
X40N     14.87
X40      12.63
XY25     10.98
XR        9.05
AR        8.30
OX40N     7.85
X5K       2.23
MH        1.72
X200      1.39
SR        1.21
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.47
H-SC    27.36
H-MC    19.24
M-SC    12.15
M-LC     7.12
M-MC     2.61
L-SC     1.47
L-LC     1.05
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.67,-0.36,33.94
IT,11.80,-15.80,72.86
FINANCE,8.41,-16.44,58.69
BANKS,7.48,-14.36,61.02
PAINTS,5.72,-14.77,37.39
ELECTRICAL,5.45,-1.49,49.86
MISC,5.33,-22.11,85.40
HEALTHCARE,4.64,2.03,23.99
AUTO,4.34,-17.14,68.11


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2761626.0,25
XR,1088819.0,13
AR,1042982.0,10
X40,772695.0,10
X40N,572055.0,12
XY25,480297.0,8
OX40N,332539.0,10
SR,182024.0,2
X5K,142389.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2683414.0,30
M-SC,1353155.0,15
H-LC,1212320.0,20
H-MC,1145664.0,15
M-LC,431635.0,5
M-MC,330972.0,2
L-SC,266398.0,3
L-MC,56986.0,1
L-LC,49758.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      922941.0      7
M-SC       XY24      836437.0      7
H-SC       AR        611408.0      4
           XR        514181.0      6
H-LC       X40       506094.0      6
H-MC       XY24      445799.0      4
M-MC       XY24      330972.0      2
H-LC       X40N      272939.0      6
H-SC       X40N      223888.0      4
H-LC       AR        215898.0      2
H-MC       XY25      205442.0      2
           X40       192065.0      3
H-SC       SR        182024.0      2
L-SC       XR        179479.0      2
M-LC       XY24      178603.0      2
H-SC       OX40N     175351.0      6
M-SC       AR        157502.0      2
M-LC       X5K       131001.0      1
H-MC       XR        118772.0      1
M-SC       XY25      118292.0      1
           XR        113504.0      2
M-LC       XR        105897.0      1
H-LC       XY25       90671.0      3
L-SC       OX40N      86919.0      1
H-LC       X200       75988.0      1
H-MC       X40N       75228.0      2
M-SC       X40        74536.0      1
H-MC       AR         58174.0      1
L-MC       XR         56986.0      1
H-SC       MH         53621.0      1
M-SC       OX40N      52884.0      2
L-LC       XY25       49758.0      1
H-LC       XY24       39342.0      1
H-MC       MH         32799.0      1
           OX40N      17385.0      1
M-LC       XY25       16134.0      1
H-LC       X5K        11388.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
